In [1]:
import gpflow as gpf
import numpy as np
import gpflow.multioutput.kernels as mk
import gpflow.multioutput.features as mf
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from time import time
import tensorflow as tf
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import os
from link_bot_notebooks import experiments_util
from link_bot_gaussian_process.link_bot_gp import LinkBotGP
from link_bot_gaussian_process import data_reformatting, error_metrics
from tabulate import tabulate
tf.logging.set_verbosity(tf.logging.FATAL)
np.set_printoptions(suppress=True, precision=4, linewidth=200)

In [2]:
plt.style.use("slides")
plt.style.use("slides")

In [56]:
def format_inverse_data2(data, traj_idx_start=0, traj_idx_end=-1, examples_per_traj=5):
    """ this assumes trajectories have one constants control input """
    states = data['states'][traj_idx_start:traj_idx_end]
    actions = data['actions'][traj_idx_start:traj_idx_end]

    n_traj, max_n_steps, n_state = states.shape
    start_indeces = np.random.randint(1, max_n_steps, size=(n_traj * examples_per_traj))
    end_indeces = np.random.randint(1, max_n_steps, size=(n_traj * examples_per_traj))
    for i in np.argwhere(start_indeces > end_indeces):
        # https://stackoverflow.com/questions/14836228/is-there-a-standardized-method-to-swap-two-variables-in-python
        # yes, this is correct.
        start_indeces[i], end_indeces[i] = end_indeces[i], start_indeces[i]
    for i in np.argwhere(start_indeces == end_indeces):
        if end_indeces[i] == max_n_steps - 1:
            start_indeces[i] -= 1
        else:
            end_indeces[i] += 1

    traj_indeces = np.repeat(np.arange(n_traj), examples_per_traj)
    delta = states[traj_indeces, end_indeces] - states[traj_indeces, start_indeces]

    delta_flat = delta.reshape(-1, 6)
    head_delta_mag = np.linalg.norm(delta_flat[:, 4:6], axis=1, keepdims=True)
    actions_flat = actions[traj_indeces, 0]
    num_steps_flat = (end_indeces - start_indeces).reshape(-1, 1)
    mag_flat = np.linalg.norm(actions_flat, axis=1).reshape(-1, 1)
    actions_flat_scaled = actions_flat / np.linalg.norm(actions_flat, axis=1).reshape(-1, 1)
    # the action representation here is cos(theta), sin(theta), magnitude
    # I think this is better than predicting just components or mag/theta
    # because theta is discontinuous and GPs assume smoothness
    x = np.concatenate((delta_flat, head_delta_mag), axis=1)
    y = np.concatenate((actions_flat_scaled, mag_flat, num_steps_flat), axis=1)

    return x, y

In [57]:
inv_train_data = format_inverse_data2(data, train_idx_start, train_idx_end)
inv_train_x = inv_train_data[0]
inv_train_y = inv_train_data[1]
print(inv_train_x[:10])
print(inv_train_y[:10])

[[ 0.1063  0.0107  0.0007  0.3419 -0.0005  0.3429  0.3429]
 [ 0.1155  0.0114  0.002   0.4281 -0.0003  0.43    0.43  ]
 [ 0.2837 -0.0033  0.3172  0.8384  0.0291  1.2297  1.23  ]
 [ 0.7297  0.2957  0.2058  1.6358  0.0184  1.8392  1.8393]
 [ 0.6014  0.2264  0.003   1.0537 -0.      1.0562  1.0562]
 [-0.0478  0.334  -0.4     0.1182 -0.3032  0.2238  0.3769]
 [-0.0008  0.0264 -0.023   0.0178 -0.0229  0.0178  0.029 ]
 [-0.0002  0.0367 -0.0025  0.036  -0.0241  0.0155  0.0286]
 [-0.0019  0.02   -0.0387  0.0014 -0.0238  0.0193  0.0306]
 [-0.0021  0.0226 -0.04    0.0014 -0.0243  0.0193  0.0311]]
[[ 0.217   0.9762  0.5488  8.    ]
 [ 0.217   0.9762  0.5488 10.    ]
 [ 0.217   0.9762  0.5488 28.    ]
 [ 0.217   0.9762  0.5488 43.    ]
 [ 0.217   0.9762  0.5488 25.    ]
 [-0.7776  0.6288  0.4376 13.    ]
 [-0.7776  0.6288  0.4376  1.    ]
 [-0.7776  0.6288  0.4376  1.    ]
 [-0.7776  0.6288  0.4376  1.    ]
 [-0.7776  0.6288  0.4376  1.    ]]


In [58]:
data = np.load("240_50_random4.npz")
train_idx_start = 0
train_idx_end = 200
test_idx_start = 200
test_idx_end = 240

fwd_train_data = data_reformatting.format_forward_data(data, train_idx_start, train_idx_end)
fwd_train_x_flat, fwd_train_y, fwd_train_u_flat, fwd_train_x, fwd_train_x_trajs, fwd_train_u_trajs = fwd_train_data
fwd_test_data = data_reformatting.format_forward_data(data, test_idx_start, test_idx_end)
fwd_test_x_flat, fwd_test_y, fwd_test_u_flat, fwd_test_x, fwd_test_x_trajs, fwd_test_u_trajs = fwd_test_data

inv_train_data = format_inverse_data2(data, train_idx_start, train_idx_end)
inv_train_x = inv_train_data[0]
inv_train_y = inv_train_data[1]
inv_test_data = format_inverse_data2(data, test_idx_start, test_idx_end)
inv_test_x = inv_test_data[0]
inv_test_y = inv_test_data[1]

In [62]:
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=False, per_process_gpu_memory_fraction=0.1))
gpf.reset_default_session(config=config)
fwd_model = LinkBotGP()
inv_model = LinkBotGP()

fwd_model.train(fwd_train_x, fwd_train_y, maximum_training_iterations=300, n_inducing_points=10)
inv_model.train(inv_train_x, inv_train_y, maximum_training_iterations=300, n_inducing_points=10)

training time: 38.37777233123779s
training time: 18.609022855758667s


In [63]:
log_path = experiments_util.experiment_name('separate_independent', 'gpf')
fwd_model.save(log_path, 'fwd_model')
inv_model.save(log_path, 'inv_model')

Saving model to /home/pmitrano/catkin_ws/src/link_bot/link_bot_gaussian_process/notebooks/log_data/separate_independent/May_31_18-03-43__bf505bde20__gpf/fwd_model
Saving model to /home/pmitrano/catkin_ws/src/link_bot/link_bot_gaussian_process/notebooks/log_data/separate_independent/May_31_18-03-43__bf505bde20__gpf/inv_model


In [64]:
headers = ['error metric', 'min', 'max', 'mean', 'median', 'std']
aggregate_metrics = np.vstack((error_metrics.fwd_model_error_metrics(fwd_model, fwd_test_x, fwd_test_y),
                               error_metrics.inv_model_error_metrics(inv_model, inv_test_x, inv_test_y)))
table = tabulate(aggregate_metrics, headers=headers, tablefmt='github', floatfmt='6.3f')
print(table)

| error metric             |    min |     max |   mean |   median |    std |
|--------------------------|--------|---------|--------|----------|--------|
| tail position error (m)  |  0.000 |   0.091 |  0.020 |    0.014 |  0.018 |
| mid position error (m)   |  0.000 |   0.087 |  0.018 |    0.011 |  0.018 |
| head position error (m)  |  0.000 |   0.058 |  0.005 |    0.005 |  0.003 |
| total position error (m) |  0.005 |   0.169 |  0.043 |    0.032 |  0.032 |
| speed (m/s)              |  0.000 |   0.704 |  0.192 |    0.159 |  0.161 |
| angle (deg)              |  0.017 | 123.466 | 15.613 |   10.172 | 16.873 |
| time steps               |  0.072 |  23.366 |  6.811 |    7.030 |  3.949 |


In [66]:
pred_y = inv_model.model.predict_y(inv_test_x)[0]
print(inv_model.model.predict_y(inv_test_x[:10])[0])
print(np.max(inv_test_x[:, 3]))

[[-0.2937  0.0983  0.2885 11.5261]
 [ 0.0133 -0.0592  0.2556 10.7725]
 [-0.8878  0.4407  0.3799 13.6736]
 [-0.8796  0.7319  0.5178 17.1982]
 [-0.9876  0.5637  0.4269 14.8194]
 [-0.4585  0.8119  0.7034 23.2311]
 [-0.6943  0.9977  0.4688 15.7133]
 [-0.6796  1.0041  0.4708 15.7465]
 [-0.3972  0.8191  0.7212 23.1459]
 [-0.4682  0.673   0.3708 13.3433]]
2.5900835448556085


In [53]:
print(pred_y[:, 3])

[15.8476 13.9709 11.9784 16.0009 12.533  14.6802 16.6578 20.5387 14.2149 18.4198 28.7587 20.1584 27.5322 27.3138 28.0619 20.1163 17.5867 23.4253 30.2449 18.1358 25.803  18.3925 22.1562 14.3646
 26.9416 12.0512 12.661  11.9386 13.3199 13.9508 13.0564 12.2449 12.2246 12.5368 27.2607 14.6251 15.6519 20.3891 16.951  24.2075 26.6258 24.1772 17.2405 21.016  16.5319 14.9099 26.1676 13.3873
 13.4172 22.1503 13.3085 14.7432 13.116  15.288  13.1866 12.3337 14.7804 28.0985 16.538  22.2211 29.39   18.3385 24.0295 13.8929 19.8235 24.0749 29.207  15.7692 15.2029 23.7053 16.422  18.3416
 14.3857 17.0854 13.1258 12.1861 14.4205 12.6347 11.9847 11.967  19.4527 21.7772 19.477  21.7833 13.6795 14.527  14.879  16.1995 12.9003 17.4778 15.5661 26.6871 14.8429 26.3579 12.6861 23.4427
 30.638  29.9074 12.5839 31.2332 20.2655 28.12   13.4489 15.4485 13.3947 25.9973 25.7606 14.0153 12.6484 19.277  28.069  14.3488 18.2828 25.7044 31.0006 13.5881 15.5521 13.7709 12.1743 13.6042
 24.3793 25.9941 14.2547 26.6715 13

In [ ]:
untrained_A = np.zeros(2)
untrained_B = np.eye(2)

trained_A = np.array([[0.005394, -0.002816], [0.001254, -0.00443]])
trained_B = np.array([[0.366756,  0.017419], [0.023738, 0.378886]])

# FIXME: This are probabily wrong now
def predict_linear_model(A, B, x_traj, u_traj):
    dt = 0.1
    initial_x = x_traj[0][0:2]  # select just the tail
    xs = np.ndarray((u_traj.shape[0], 2))
    xs[0] = initial_x
    x = initial_x
    for i, u in enumerate(u_traj):
        x = x + dt*A@x + dt*B@u
        xs[i] = x
    return xs

def one_step_predict_linear_model(A, B, x_traj, u_traj):
    dt = 0.1
    xs = np.ndarray((u_traj.shape[0], 2))
    xs[0] = x_traj[0][0:2]
    for i in range(u_traj.shape[0] - 1):
        u = u_traj[i]
        x = x_traj[i][0:2]
        x_next = x + dt*A@x + dt*B@u
        xs[i+1] = x_next
    return xs

def batch_one_step_predict_linear_model(A, B, batch_x_traj, batch_u_traj):
    n_traj, n_step, _  = batch_x_traj.shape
    xs = np.ndarray((n_traj, n_step, 2))
    for i, (x_traj, u_traj) in enumerate(zip(batch_x_traj, batch_u_traj)):
        xs[i] = one_step_predict_linear_model(A, B, x_traj, u_traj)
    return xs

In [ ]:
total_displacement = np.linalg.norm(fwd_test_y[:, 0:2], axis=1) + np.linalg.norm(fwd_test_y[:, 2:4], axis=1) + np.linalg.norm(fwd_test_y[:, 4:6], axis=1)
print("Some stats about the testing data:")
print('min displacement in test', np.min(total_displacement))
print('max displacement in test', np.max(total_displacement))
print('mean displacement in test', np.mean(total_displacement))

In [ ]:
# error for linear models
untrained_linear_data = batch_one_step_predict_linear_model(untrained_A, untrained_B, fwd_test_x_trajs, fwd_test_u_trajs)
trained_linear_data = batch_one_step_predict_linear_model(trained_A, trained_B, fwd_test_x_trajs, fwd_test_u_trajs)

untrained_linear_tail_error = np.linalg.norm(untrained_linear_data - fwd_test_x_trajs[:, :, :2], axis=2)
trained_linear_tail_error = np.linalg.norm(trained_linear_data - fwd_test_x_trajs[:, :, :2], axis=2)
print("Stats about the old linear models:")
print("untrained linear model tail position error (min/max/mean/median)", untrained_linear_tail_error.min(), untrained_linear_tail_error.max(), untrained_linear_tail_error.mean(), np.median(untrained_linear_tail_error), np.std(untrained_linear_tail_error))
print("trained linear model tail position error (min/max/mean/median)", trained_linear_tail_error.min(), trained_linear_tail_error.max(), trained_linear_tail_error.mean(), np.median(trained_linear_tail_error), np.std(trained_linear_tail_error))
print(np.unravel_index(untrained_linear_tail_error.argmax(), untrained_linear_tail_error.shape))

In [ ]:
def predict(fwd_model, traj, steps=1, initial_variance = 0.00001):
    test_x_traj, test_u_traj  = traj
    traj_length = test_x_traj.shape[0]
    
    assert steps < traj_length, "steps {} > traj length {}!".format(steps, traj_length)
    
    if test_x_traj.shape[0] > test_u_traj.shape[0]:
        test_x_traj = test_x_traj[:test_u_traj.shape[0]]
        
    mu_combined_test_x = np.hstack((test_x_traj, test_u_traj))
        
    # define the initial distribution
    mu_combined_test_x_t = mu_combined_test_x[0]
    sigma_combined_test_x = np.eye(fwd_model.n_inputs) * initial_variance
    # assume no control variance
    sigma_combined_test_x[-2, -2] = 0
    sigma_combined_test_x[-1, -1] = 0
    
    # sample from that initial distribution to get initial particles to feed into the GP
    num_particles = 50
    combined_x_t_particles = np.random.multivariate_normal(mu_combined_test_x_t, sigma_combined_test_x, num_particles)
    particles = np.zeros((steps, num_particles, fwd_model.n_inputs))
    
    for t in range(steps):
        particles[t] = combined_x_t_particles
        combined_x_t_particles_relative = data_reformatting.make_relative_to_head(combined_x_t_particles)
        mu_delta_x_t_plus_1s, var_delta_x_t_plus_1s = fwd_model.model.predict_y(combined_x_t_particles_relative)
        # sample point from the gaussian prediction
        combined_x_t_plus_1_particles = np.empty_like(combined_x_t_particles)
        for j, (mu_delta_x_t_plus_1_j, var_delta_x_t_plus_1_j) in enumerate(zip(mu_delta_x_t_plus_1s, var_delta_x_t_plus_1s)):
            # We assumed that the GPs are independant for each output dimension, so the full covariance matrix is diagonal
            sigma_delta_x_t_plus_1_j = np.diag(var_delta_x_t_plus_1_j)
            u_t_plus_1_j = test_u_traj[t + 1]
            delta_delta_x_t_j = np.random.multivariate_normal(mu_delta_x_t_plus_1_j, sigma_delta_x_t_plus_1_j)
            # predict only gives the delta position, so we have to integrate here
            delta_delta_combined_x_t_j = np.hstack((delta_delta_x_t_j, [0, 0]))
            combined_x_t_plus_1_particles[j] =  combined_x_t_particles[j] + delta_delta_combined_x_t_j
        
        combined_x_t_particles = combined_x_t_plus_1_particles
            
    return particles, test_x_traj, test_u_traj

In [ ]:
def predict_no_ground_truth(my_model, initial_x, u, steps=1, initial_variance = 0.00001):
    # define the initial distribution
    mu_combined_x_t =  np.hstack((initial_x, u))
    sigma_combined_x_t = np.eye(my_model.n_inputs) * initial_variance
    # assume no control variance
    sigma_combined_x_t[-2, -2] = 0
    sigma_combined_x_t[-1, -1] = 0
    
    # sample from that initial distribution to get initial particles to feed into the GP
    num_particles = 50
    combined_x_t_particles = np.random.multivariate_normal(mu_combined_x_t, sigma_combined_x_t, num_particles)
    particles = np.zeros((steps, num_particles, my_model.n_inputs))
    
    x_traj = np.reshape(initial_x, [1, -1])
    u_traj = np.ones((steps, 2))*u
    trained_linear_data = predict_linear_model(trained_A, trained_B, x_traj, u_traj)

    for t in range(steps):
        particles[t] = combined_x_t_particles
        combined_x_t_particles_relative = data_reformatting.make_relative_to_head(combined_x_t_particles)
        mu_delta_x_t_plus_1s, var_delta_x_t_plus_1s = fwd_model.model.predict_y(combined_x_t_particles_relative)
        # sample point from the gaussian prediction
        combined_x_t_plus_1_particles = np.empty_like(combined_x_t_particles)
        for j, (mu_delta_x_t_plus_1_j, var_delta_x_t_plus_1_j) in enumerate(zip(mu_delta_x_t_plus_1s, var_delta_x_t_plus_1s)):
            # We assumed that the GPs are independant for each output dimension, so the full covariance matrix is diagonal
            sigma_delta_x_t_plus_1_j = np.diag(var_delta_x_t_plus_1_j)
            delta_delta_x_t_j = np.random.multivariate_normal(mu_delta_x_t_plus_1_j, sigma_delta_x_t_plus_1_j)
            # predict only gives the delta position, so we have to integrate here
            delta_delta_combined_x_t_j = np.hstack((delta_delta_x_t_j, [0, 0]))
            combined_x_t_plus_1_particles[j] =  combined_x_t_particles[j] + delta_delta_combined_x_t_j
        
        combined_x_t_particles = combined_x_t_plus_1_particles
            
    return particles, trained_linear_data

In [ ]:
%matplotlib inline 

def plot_predict(particles, test_x_traj=None, test_u_traj=None, untrained_linear_data=None, trained_linear_data=None):
    T = particles.shape[0]
    if test_x_traj is not None:
        x0s = [test_x_traj[0, 0], test_x_traj[0, 2], test_x_traj[0, 4]]
        y0s = [test_x_traj[0, 1], test_x_traj[0, 3], test_x_traj[0, 5]]
    
    fig = plt.figure(figsize=(10,10))
    
    x_t_particles_data = []
    for x_t_particle in particles[0]:
        x_t_particles_xs = [x_t_particle[0], x_t_particle[2], x_t_particle[4]]
        x_t_particles_ys = [x_t_particle[1], x_t_particle[3], x_t_particle[5]]
        line = plt.plot(x_t_particles_xs, x_t_particles_ys, color='black', alpha=0.2)[0]
        x_t_particles_data.append(line)
            
    if test_x_traj is not None:
        plt.plot(x0s, y0s, color='orange')
        
    x_next_data = []
    if test_x_traj is not None:
        x_t_plus_1 = test_x_traj[1]
        xs_next = [x_t_plus_1[0], x_t_plus_1[2], x_t_plus_1[4]]
        ys_next = [x_t_plus_1[1], x_t_plus_1[3], x_t_plus_1[5]]
        x_next_data = plt.plot(xs_next, ys_next, color='blue')[0]
        
    untrained_linear_x_next_data = []
    if untrained_linear_data is not None:
        untrained_linear_x_t_plus_1 = untrained_linear_data[1]
        untrained_linear_x_next_data = plt.scatter(untrained_linear_x_t_plus_1[0], untrained_linear_x_t_plus_1[1], color='r')
    trained_linear_x_next_data = []
    if trained_linear_data is not None:
        trained_linear_x_t_plus_1 = trained_linear_data[1]
        trained_linear_x_next_data = plt.scatter(trained_linear_x_t_plus_1[0], trained_linear_x_t_plus_1[1], color='g')

    if test_x_traj is not None:
        plt.quiver(test_x_traj[0, 4], test_x_traj[0, 5], test_u_traj[0, 0], test_u_traj[0, 1], color='black', width=0.004)

    plt.xlabel("x (m)")
    plt.ylabel("y (m)")
    plt.xlim([-7,7])
    plt.ylim([-7,7])
    custom_lines = [Line2D([0], [0], color='black', lw=1, alpha=0.1),
                    Line2D([0], [0], color='orange', lw=1),
                    Line2D([0], [0], color='blue', lw=1),
                    Line2D([0], [0], color='red', lw=1),
                    Line2D([0], [0], color='green', lw=1),
                   ]

    l = plt.legend(custom_lines, ['predictions', r'$x_0$', r'$x_t$', 'linear (init)', 'linear (fine-tuned)'])
    
    def update(t):
        x_t_particles = particles[t]
        for x_t_particle_d, x_t_particle in zip(x_t_particles_data, x_t_particles):
            x_t_particles_xs = [x_t_particle[0], x_t_particle[2], x_t_particle[4]]
            x_t_particles_ys = [x_t_particle[1], x_t_particle[3], x_t_particle[5]]
            x_t_particle_d.set_xdata(x_t_particles_xs)
            x_t_particle_d.set_ydata(x_t_particles_ys)
            
        if untrained_linear_data is not None:
            untrained_linear_x_t_plus_1 = untrained_linear_data[t]
            untrained_linear_x_next_data.set_offsets([untrained_linear_x_t_plus_1[0],  untrained_linear_x_t_plus_1[1]])
        if untrained_linear_data is not None:
            trained_linear_x_t_plus_1 = trained_linear_data[t]
            trained_linear_x_next_data.set_offsets([trained_linear_x_t_plus_1[0],  trained_linear_x_t_plus_1[1]])
        
        if test_x_traj is not None:
            x_t_plus_1 = test_x_traj[t]
            xs_next = [x_t_plus_1[0], x_t_plus_1[2], x_t_plus_1[4]]
            ys_next = [x_t_plus_1[1], x_t_plus_1[3], x_t_plus_1[5]]
            x_next_data.set_xdata(xs_next)
            x_next_data.set_ydata(ys_next)
        ax = fig.gca()
        ax.relim()
        ax.autoscale_view()

    
    anim = FuncAnimation(fig, update, frames=np.arange(0, T), interval=100)
    plt.close()
    return anim

In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

In [ ]:
np.random.seed(1)
# 1, 9 and 13 are interesting
# 8 and 22 are the worst cases for the old linear models
# 14 is accurate
# 3 is really bad
trajectory_idx = 23
traj = fwd_test_x_trajs[trajectory_idx], fwd_test_u_trajs[trajectory_idx]
particles, fwd_test_x_traj, fwd_test_u_traj = predict(fwd_model, traj, steps=49)
untrained_linear_data = predict_linear_model(untrained_A, untrained_B, fwd_test_x_trajs[trajectory_idx], fwd_test_u_trajs[trajectory_idx])
trained_linear_data = predict_linear_model(trained_A, trained_B, fwd_test_x_trajs[trajectory_idx], fwd_test_u_trajs[trajectory_idx])
anim = plot_predict(particles, fwd_test_x_traj, fwd_test_u_traj, untrained_linear_data, trained_linear_data)
anim_html = anim.to_jshtml()
anim.save('test_ex_{}.gif'.format(trajectory_idx), writer='imagemagick', fps=20)
HTML(anim.to_jshtml())

In [ ]:
x = np.random.randn(3,3, 3)
print(x)

In [ ]:
np.tile(np.arange(10), [3])
# np.repeat(np.arange(10), 3)

In [ ]:
# initial_x = np.array([-1, 0, 0, 0, 1, 0])
# u = np.array([0.0, -0.5])
initial_x = fwd_test_x_trajs[23, 0]
u = fwd_test_u_trajs[23, 0]
print(initial_x, u)
particles, linear_data = predict_no_ground_truth(fwd_model, initial_x, u, steps=176)
anim = plot_predict(particles, trained_linear_data=linear_data)
HTML(anim.to_jshtml())

In [ ]:
particles[-1]

In [ ]:
initial_x = fwd_test_x_trajs[23, 0]
u = fwd_test_u_trajs[23, 0]
v1 = np.array([initial_x[3] - initial_x[1], initial_x[2]-initial_x[0]])    
v2 = np.array([initial_x[5] - initial_x[3], initial_x[4]-initial_x[2]])    
np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1)*np.linalg.norm(v1)))

In [ ]:
def plot_inv_prediction(inv_x, inv_y):
    inv_x = np.expand_dims(inv_x, axis=0)
    mu, sig = inv_model.model.predict_y(inv_x)
    s1 = np.array([[0, 0, 0, 0, 0, 0]])
    s2 = s1 + inv_x

    plt.figure(figsize=(10,10))
    plt.scatter(0, 0, color='m', s=100)
    plt.scatter(s2[0,4], s2[0,5], color='b', s=1000)
    mu_cos = mu[0, 0]
    lower_bound_cos = mu_cos - 1.96 * sig[0, 0]
    upper_bound_cos = mu_cos + 1.96 * sig[0, 0]
    mu_sin = mu[0, 1]
    lower_bound_sin = mu_sin - 1.96 * sig[0, 1]
    upper_bound_sin = mu_sin + 1.96 * sig[0, 1]
    mu_mag = mu[0, 2]
    mu_nu = np.linalg.norm(mu[0, :2])
    lower_bound_nu = np.hypot(lower_bound_cos, lower_bound_sin)
    upper_bound_nu = np.hypot(upper_bound_cos, upper_bound_sin)
    plt.quiver([s1[0,4], s1[0,4], s1[0,4], s1[0,4]],
               [s1[0,5], s1[0,5], s1[0,5], s1[0,5]],
               [lower_bound_cos / lower_bound_nu * mu_mag, upper_bound_cos / upper_bound_nu * mu_mag, mu_cos / mu_nu * mu_mag, inv_y[0] * inv_y[2]],
               [lower_bound_sin / lower_bound_nu * mu_mag, upper_bound_sin / upper_bound_nu * mu_mag, mu_sin / mu_nu * mu_mag, inv_y[1] * inv_y[2]],
               color=['r', 'r', 'g', 'k'],
               width=0.005,
               scale=1.5,
              )
    plt.axis("equal")
    custom_lines = [Line2D([0], [0], color='red', lw=1),
                    Line2D([0], [0], color='green', lw=1),
                    Line2D([0], [0], color='black', lw=1),
                   ]

    l = plt.legend(custom_lines, ['95% confidence bounds', 'mean', 'true'])
    
for i in range(10):
    test_idx = 50 * i
    plot_inv_prediction(inv_test_x[test_idx], inv_test_y[test_idx])

In [ ]:
mu, sig = inv_model.model.predict_y(test_inverse_combined_x)
print(np.min(mu, axis=0))

# Profiling the GP prediction

In [ ]:
%%timeit
x_star = np.random.randn(1,8)
fwd_model.model.predict_y(x_star)

In [ ]:
%%timeit
x_star = np.random.randn(1,6)
inv_model.model.predict_y(x_star)